In [72]:
import sys, os
sys.path.insert(0,"../code")

import pandas as pd
import numpy as np
import geopandas as gpd
import folium 
import math

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data 

In [73]:
def load_data(path_data="../data"): 
    
    if not os.path.exists(path_data): 
        os.mkdir(path)
    
    if not os.path.path.exists(os.path.join(path_data,"nyc-taxi-trip-duration")): 
        pass
        #!curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD
        #download file 
        #unzip file 

In [74]:
#PATH_DATA = os.path.join("..\data","nyc-taxi-trip-duration","train","train.csv")
PATH_DATA= r'D:\Tools\Git_Repo\NYC_TaxiTripDuration\data\train\train.csv'
df_rides = pd.read_csv(PATH_DATA,index_col=0,parse_dates=[2,3],dtype={'store_and_fwd_flag':'category','vendor_id':'category','passenger_count':'int8',})
print(df_rides.info())
df_rides.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1458644 entries, id2875421 to id1209952
Data columns (total 10 columns):
vendor_id             1458644 non-null category
pickup_datetime       1458644 non-null datetime64[ns]
dropoff_datetime      1458644 non-null datetime64[ns]
passenger_count       1458644 non-null int8
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null category
trip_duration         1458644 non-null int64
dtypes: category(2), datetime64[ns](2), float64(4), int64(1), int8(1)
memory usage: 93.2+ MB
None


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [75]:
pd.isna(df_rides).sum()

vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [76]:
assert(df_rides.index.unique().shape[0]==df_rides.shape[0])
assert(np.all(df_rides.dropoff_datetime>df_rides.pickup_datetime))
assert(np.all(df_rides.passenger_count>=0))
#assert(np.allclose((df_rides.dropoff_datetime-df_rides.pickup_datetime).dt.seconds.values,df_rides.trip_duration,))

In [77]:
from shapely.geometry import Point

def transform_to_geodf(df_rides,set_geometry_col='pickup'): 
        df_rides['pickup_geom'] = list(zip(df_rides.pickup_longitude,df_rides.pickup_latitude))
        df_rides['pickup_geom'] = df_rides['pickup_geom'].apply(Point)
        df_rides['dropoff_geom'] = list(zip(df_rides.dropoff_longitude,df_rides.dropoff_latitude))
        df_rides['dropoff_geom'] = df_rides['dropoff_geom'].apply(Point)
        df_rides = gpd.GeoDataFrame(df_rides,geometry=f'{set_geometry_col}_geom',crs={"init":"epsg:4326"})
        
        return df_rides

In [78]:
df_rides = transform_to_geodf(df_rides)
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom
id,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,POINT (-73.98215484619139 40.76793670654297),POINT (-73.96463012695313 40.76560211181641)
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,POINT (-73.98041534423827 40.73856353759766),POINT (-73.99948120117188 40.73115158081055)
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,POINT (-73.97902679443358 40.76393890380859),POINT (-74.00533294677734 40.71008682250977)
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,POINT (-74.01004028320313 40.719970703125),POINT (-74.01226806640625 40.70671844482422)
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,POINT (-73.97305297851563 40.79320907592773),POINT (-73.97292327880859 40.78252029418945)


In [79]:
df_rides.shape

(1458644, 12)

In [80]:
df_rides = df_rides.to_crs({'init':'epsg:32618'})

In [81]:
df_rides.head()


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom
id,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,POINT (585902.5443471848 4513494.632064121),POINT (-73.96463012695313 40.76560211181641)
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,POINT (586087.236356966 4510235.698938999),POINT (-73.99948120117188 40.73115158081055)
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,POINT (586171.7092893183 4513053.9131043),POINT (-74.00533294677734 40.71008682250977)
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,POINT (583609.1275845997 4508143.138655966),POINT (-74.01226806640625 40.70671844482422)
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,POINT (586637.8752449993 4516309.016731326),POINT (-73.97292327880859 40.78252029418945)


In [82]:
import pandas as pd 
import numpy as np
import pyproj

# Correct utm-zone: {'init': 'epsg:32618'}
#df_rides.to_crs()

def convert_projection_to_utm(df,col_x_source,col_y_source,
                              col_x_dest = 'x_utm', col_y_dest = 'y_utm',
                              projection_source=pyproj.Proj("+init=EPSG:4326"),
                              projection_dest=pyproj.Proj("+init=EPSG:32618")):
    x,y = pyproj.transform(projection_source, projection_dest,df[col_x_source].values,df[col_y_source].values)
    return df.assign(**{col_x_dest:x,col_y_dest:y})

def calc_distance(df,col_x1_utm,col_x2_utm,col_y1_utm,col_y2_utm,type_='beeline'):
    if type_ =='beeline':
        distance = np.sqrt((df[col_x1_utm]-df[col_x2_utm])**2+(df[col_y1_utm]-df[col_y2_utm])**2)
    elif type_ =='Manhattan':
        distance = np.abs((df[col_x1_utm]-df[col_x2_utm]))+np.abs((df[col_y1_utm]-df[col_y2_utm]))
    return df.assign(**{f"distance_{type_}": distance})

df_rides = convert_projection_to_utm(df_rides,col_x_source='pickup_longitude',col_y_source='pickup_latitude',col_x_dest="pickup_x_utm",col_y_dest='pickup_y_utm')
df_rides = convert_projection_to_utm(df_rides,col_x_source='dropoff_longitude',col_y_source='dropoff_latitude',col_x_dest="dropoff_x_utm",col_y_dest='dropoff_y_utm')
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom,pickup_x_utm,pickup_y_utm,dropoff_x_utm,dropoff_y_utm
id,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,POINT (585902.5443471848 4513494.632064121),POINT (-73.96463012695313 40.76560211181641),585902.544347,4.513495e+06,587384.651075,4.513253e+06
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,POINT (586087.236356966 4510235.698938999),POINT (-73.99948120117188 40.73115158081055),586087.236357,4.510236e+06,584486.788662,4.509394e+06
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,POINT (586171.7092893183 4513053.9131043),POINT (-74.00533294677734 40.71008682250977),586171.709289,4.513054e+06,584019.127278,4.507050e+06
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,POINT (583609.1275845997 4508143.138655966),POINT (-74.01226806640625 40.70671844482422),583609.127585,4.508143e+06,583437.516201,4.506670e+06
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,POINT (586637.8752449993 4516309.016731326),POINT (-73.97292327880859 40.78252029418945),586637.875245,4.516309e+06,586662.713668,4.515123e+06


In [83]:
df_rides= calc_distance(df_rides,'pickup_x_utm','dropoff_x_utm','pickup_y_utm','dropoff_y_utm')

In [84]:
df_rides= calc_distance(df_rides,'pickup_x_utm','dropoff_x_utm','pickup_y_utm','dropoff_y_utm',type_='Manhattan')

In [85]:
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom,pickup_x_utm,pickup_y_utm,dropoff_x_utm,dropoff_y_utm,distance_beeline,distance_Manhattan
id,,,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,POINT (585902.5443471848 4513494.632064121),POINT (-73.96463012695313 40.76560211181641),585902.544347,4.513495e+06,587384.651075,4.513253e+06,1501.709771,1723.957684
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,POINT (586087.236356966 4510235.698938999),POINT (-73.99948120117188 40.73115158081055),586087.236357,4.510236e+06,584486.788662,4.509394e+06,1808.098438,2441.748555
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,POINT (586171.7092893183 4513053.9131043),POINT (-74.00533294677734 40.71008682250977),586171.709289,4.513054e+06,584019.127278,4.507050e+06,6377.703936,8156.039197
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,POINT (583609.1275845997 4508143.138655966),POINT (-74.01226806640625 40.70671844482422),583609.127585,4.508143e+06,583437.516201,4.506670e+06,1483.166420,1644.816109
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,POINT (586637.8752449993 4516309.016731326),POINT (-73.97292327880859 40.78252029418945),586637.875245,4.516309e+06,586662.713668,4.515123e+06,1186.672542,1211.250987


In [86]:
def calc_time(df,pickup_datetime,dropoff_datetime):
    weekday = df[pickup_datetime].dt.dayofweek
    hour    = df[pickup_datetime].dt.hour
    quarter = df[pickup_datetime].dt.quarter
    month   = df[pickup_datetime].dt.month
    day_of_year = df[pickup_datetime].dt.dayofyear
    
    return df.assign(**{"weekday": weekday,"hour":hour,"quarter":quarter,'month':month,'day_of_year':day_of_year})


In [87]:

    
#     dr = pd.date_range(start=df[pickup_datetime].min(), end=df[dropoff_datetime].max())
#     df_times' = pd.DataFrame()
#     df_times['Date'] = dr

#     cal = calendar()
#     holidays = cal.holidays(start=dr.min(), end=dr.max())
#     holidays = df_times['Date'].isin(holidays)
#     test = df.join(holidays, how='outer')

In [88]:
df_rides = calc_time(df_rides,'pickup_datetime','dropoff_datetime')

In [89]:
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,...,pickup_y_utm,dropoff_x_utm,dropoff_y_utm,distance_beeline,distance_Manhattan,weekday,hour,quarter,month,day_of_year
id,,,,,,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,...,4.513495e+06,587384.651075,4.513253e+06,1501.709771,1723.957684,0,17,1,3,74
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,...,4.510236e+06,584486.788662,4.509394e+06,1808.098438,2441.748555,6,0,2,6,164
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,...,4.513054e+06,584019.127278,4.507050e+06,6377.703936,8156.039197,1,11,1,1,19
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,...,4.508143e+06,583437.516201,4.506670e+06,1483.166420,1644.816109,2,19,2,4,97
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,...,4.516309e+06,586662.713668,4.515123e+06,1186.672542,1211.250987,5,13,1,3,86


In [90]:
#df_rides[df_rides['holidays']==True]

In [91]:
def calc_daylight(df, pickup_latitude,day_of_year):
    P = np.arcsin(0.39795 * np.cos(0.2163108 + 2 * np.arctan(0.9671396 * np.tan(.00860 * (df[day_of_year].astype(int) - 186)))))
    #assert(False)
    pi = math.pi
    daylightamount = 24 - (24 / pi) * np.arccos(
        (np.sin((0.8333 * pi / 180) + np.sin(df[pickup_latitude] * pi / 180) * np.sin(P)) / (np.cos(df[pickup_latitude] * pi / 180) * np.cos(P))))
    #assert(False)
    return df.assign(**{"daylighthours": daylightamount})

In [92]:
%%time
df_rides = calc_daylight(df_rides,'pickup_latitude','day_of_year')

Wall time: 403 ms


In [112]:
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,...,distance_Manhattan,weekday,hour,quarter,month,day_of_year,daylighthours,Holiday,Is_on_Holiday,just_date
id,,,,,,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,...,1723.957684,0,17,1,3,74,11.864576,False,False,2016-03-14
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,...,2441.748555,6,0,2,6,164,15.017241,False,False,2016-06-12
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,...,8156.039197,1,11,1,1,19,9.675665,False,False,2016-01-19
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,...,1644.816109,2,19,2,4,97,12.900760,False,False,2016-04-06
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,...,1211.250987,5,13,1,3,86,12.409669,False,False,2016-03-26


In [136]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
def holidays(df,pickup_datetime,dropoff_datetime):
        dr = pd.date_range(start=df[pickup_datetime].min().year, end=df[pickup_datetime].max())
        df_times = pd.DataFrame()
        df_times['Date'] = dr
        cal = calendar()
        holidays = cal.holidays(start=dr.min(), end=dr.max())
        df['Holiday'] = df[pickup_datetime].dt.date.astype('datetime64').isin(holidays)
        print(holidays)
        return df.assign(**{"Is_on_Holiday": df['Holiday']})

In [137]:
df_rides = holidays(df_rides,'pickup_datetime','dropoff_datetime')

DatetimeIndex(['1970-02-16', '1970-05-25', '1970-07-03', '1970-09-07',
               '1970-10-12', '1970-11-11', '1970-11-26', '1970-12-25',
               '1971-01-01', '1971-02-15',
               ...
               '2015-07-03', '2015-09-07', '2015-10-12', '2015-11-11',
               '2015-11-26', '2015-12-25', '2016-01-01', '2016-01-18',
               '2016-02-15', '2016-05-30'],
              dtype='datetime64[ns]', length=447, freq=None)


In [138]:
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,...,distance_Manhattan,weekday,hour,quarter,month,day_of_year,daylighthours,Holiday,Is_on_Holiday,just_date
id,,,,,,,,,,,,,,,,,,,,,
id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,...,1723.957684,0,17,1,3,74,11.864576,False,False,2016-03-14
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,...,2441.748555,6,0,2,6,164,15.017241,False,False,2016-06-12
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,...,8156.039197,1,11,1,1,19,9.675665,False,False,2016-01-19
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,...,1644.816109,2,19,2,4,97,12.900760,False,False,2016-04-06
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,...,1211.250987,5,13,1,3,86,12.409669,False,False,2016-03-26


In [126]:
df_rides['Is_on_Holiday'].value_counts()

False    1431393
True       27251
Name: Is_on_Holiday, dtype: int64

In [60]:
# Correct utm-zone: {'init': 'epsg:32618'}


Jahreszeit 

Wochentag

Tageszeit

Tageslänge - Phil berechnet den Azimuthwinkel 

Feiertag

Aufteilung in 100x100 Meter Quadrate: 
- Gruppieren Mittelwert aller

dict_test = {0:'winter',1:'spring',2:'summer',3:'fall'}
df_rides.replace({"season": dict_test}).head()